In [22]:
import os
from os import path
import sqlite3
global_sqlite_filename='apf.db'
import pandas
from datetime import datetime, timedelta
import numpy as np
from astropy.time import Time

In [3]:
class col_types:
    def __init__(self):
        self.TEXT = 'TEXT'
        self.INTEGER = 'INTEGER'
        self.REAL = 'REAL'
        self.BLOB = 'BLOB'
        self.NULL = 'NULL'
        
ct = col_types()
print(ct.TEXT)

#print(os.getcwd())
#print(os.listdir())

TEXT


In [44]:
def create_database(sqlite_filename=global_sqlite_filename):
    conn = sqlite3.connect(sqlite_filename)
    cursor = conn.cursor()
    return cursor
       
def get_telemetry_columns():
    columns = {'DateTime': ct.TEXT, 
                  'TARGET': ct.NULL,
                  'MIDPTFIN': ct.REAL,
                  'AZ': ct.REAL,
                  'EL': ct.REAL,
                  'AZENCPOS': ct.REAL,
                  'ELENCPOS': ct.REAL,
                  'AZENCVEL': ct.REAL,
                  'ELENCVEL': ct.REAL,
                  'AZFLWERR': ct.REAL,
                  'ELFLWERR': ct.REAL,
                  'OUTFILE': ct.REAL,
                  'OBSNUM': ct.REAL,
                  'MODE': ct.REAL,
                  'AVG_FWHM': ct.REAL,
                  'M5WIND': ct.REAL,
                  'M5WINDAZ': ct.REAL,
                  'TAVERAGE': ct.REAL,
                  'TM1S210': ct.REAL,
                  'TM2CAIR': ct.REAL,
                  'OFFSET_AZ': ct.REAL,
                  'OFFSET_EL': ct.REAL,
                  'RMSOFFSET_AZ': ct.REAL,
                  'RMSOFFSET_EL': ct.REAL,
                  'AVGOFFSET_AZ': ct.REAL,
                  'AVGOFFSET_EL': ct.REAL,
                  'HATCHPOS': ct.TEXT,
                  'EVENT': ct.TEXT}    
    return columns
    
def get_velocity_columns():
    columns = [('MdptJulian', ct.REAL),
               ('velocity', ct.REAL),
               ('velocity_error', ct.REAL),
               ('stellar_activity1', ct.REAL),
               ('stellar_activity2', ct.REAL),
               ('total_counts', ct.REAL),
               ('brad_doesnt_know', ct.REAL),
               ('DateTime', ct.TEXT),
               ('starID', ct.TEXT)
              ]
    return columns
    
def create_telemetry_table(cursor):        
    columns = get_telemetry_columns()
    l = ['{} {}'.format(k,v) for (k, v) in columns.items()]
    print(len(l))
    full_list = ', '.join(l)
    sql_create_string = 'CREATE TABLE telemetry (' + full_list + ', PRIMARY KEY (DateTime))'
    print(sql_create_string)        
    cursor.execute(sql_create_string)

    return


def create_velocity_table(cursor):
    columns = get_velocity_columns()
    l = ['{} {}'.format(k,v) for (k, v) in columns.items()]
    full_list = ', '.join(l)
    sql_create_string = 'CREATE TABLE velocity (' + full_list + ', PRIMARY KEY (DateTime))'
    print(sql_create_string)  
    cursor.execute(sql_create_string)

    return

def open_connection(sqlite_filename=global_sqlite_filename):
    dburi = 'file:{}?mode=rw'.format(sqlite_filename)
    conn = sqlite3.connect(dburi, uri=True)
    return conn
    

In [13]:
conn = open_connection()
cursor = conn.cursor()
#create_telemetry_table(cursor)
create_velocity_table(cursor)

CREATE TABLE velocity (MdptJulian REAL, velocity REAL, velocity_error REAL, stellar_activity1 REAL, stellar_activity2 REAL, total_counts REAL, brad_doesnt_know REAL, DateTime TEXT, starID TEXT, PRIMARY KEY (DateTime))


In [8]:
def populate_telemetry_table(conn, filenames):
    for csv_file in filenames:
        try:
            df = pandas.read_csv(csv_file, 
                               sep='\t', skiprows=1, header=None, names=get_telemetry_columns(), comment='#')
            df.to_sql('telemetry', conn, if_exists='append', index=False)
            
        except Exception as e:
            print(csv_file)
            print(e)

def populate_telemetry(conn=None, dirname=None, batchsize=5):
    filenames = [os.path.join(dirname, x) for x in os.listdir(dirname)]
    filenames.sort()
    
    def batch(iterable, n=1):
        l = len(iterable)
        for ndx in range(0, l, n):
            yield iterable[ndx : min(ndx + n, l)]
    
    for file_batch in batch(filenames, n=batchsize):
        populate_telemetry_table(conn, file_batch)
        
    


        

In [14]:
#populate_telemetry(conn, '../telemetry_data', batchsize=10)

In [99]:
def load_velocity_csv(dirname, filename):
    full_path = os.path.join(dirname, filename)
    columns = get_velocity_columns()[:-2] # leave off the last two column names
                                        # because we're calculating them below.
    df = pandas.read_csv(full_path, sep='\s+',
            names=[x[0] for x in columns]) 
                                                
    return df
    

def convert_julian_to_datetime(juliantimes):
    times = Time(juliantimes, format='jd')
    return times.isot 
    
def add_calculated_columns(df, filename):
    new_df = df.copy()
    datetimes = convert_julian_to_datetime(new_df.MdptJulian)
    new_df['DateTime'] = datetimes
    new_df['starID'] = [filename] * len(new_df.index) 
    
    return new_df
    

def populate_velocity(conn, dirname='../standard_star_velocity_measurements/', debug=False):
    filenames = ['HD10700_APF.vels', 'HD185144_APF.vels', 'HD9407_APF.vels']
    for file in filenames:
        df_incomplete = load_velocity_csv(dirname, file)
        df_complete = add_calculated_columns(df_incomplete, file)
        if debug:
            print(df_complete)
        else: 
            df_complete.to_sql('velocity', conn, if_exists='append', index=False)
        print('tried to add {} entries to \'velocity\''.format(len(df_complete)))

#d = load_velocity_csv('../standard_star_velocity_measurements/', 'HD10700_APF.vels')
#add_calculated_columns(d, 'HD10700_APF.vels' )


In [102]:
populate_velocity(conn, debug=False)

tried to add 755 entries to 'velocity'
tried to add 1655 entries to 'velocity'
tried to add 694 entries to 'velocity'
